In [16]:
from docling.document_converter import DocumentConverter

source = "/home/akshay/projects/generativeAI/L05_AY2425.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source,)
print(result.document.export_to_markdown())  # output: "## Docling Technical Report[...]"

INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document L05_AY2425.pdf
INFO:docling.document_converter:Finished converting document L05_AY2425.pdf in 14.75 sec.


## Chow-Liu Trees

## Generative AI Models - Lecture 5

9 th May 2025

Thomas Krak (slides adapted from Gennaro Gala) Uncertainty in Artificial Intelligence

<!-- image -->

## Idea: structure learning as discrete optimization

- · Let X be a set of RVs and D = { x n } N n =1 be i.i.d. data
- · Let [ G ] be some family of DAGs over X
- · Define a suitable score S G D ( , )
- · Find G ∗ = arg max G∈ G [ ] S G D ( , )

- · [ G ] is the set of all directed trees [ T ] over X
- · Directed tree : Every RV has at most one parent
- · Score S G D ( , ) = max Θ L G ( , Θ , D ), where
- · Θ are all BN parameters for G (for categorical CPDs)
- · L G ( , Θ , D ) is the log-likelihood
- · Thus, tree G is 'better' than G ' if the log-likelihood of G is higher than the log-likelihood of G ' (when equipping them with their ML parameters):

<!-- image -->

- · Remarkable: Poly-time Algorithm!

## Algorithm 3 VE\_PRI(N, Q, J)

## input:

N:

Q

- variables in network N

Bayesian network

- J: ordering o

In [18]:
from pathlib import Path


result.document.save_as_markdown(filename=Path("sfs.md"))

In [14]:
import logging
from collections.abc import Iterable
from pathlib import Path


from docling_core.types.doc import DocItemLabel, DoclingDocument, NodeItem, TextItem


from docling.datamodel.base_models import InputFormat, ItemAndImageEnrichmentElement
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.models.base_model import BaseItemAndImageEnrichmentModel
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline


class ExampleFormulaUnderstandingPipelineOptions(PdfPipelineOptions):
    do_formula_understanding: bool = True

# A new enrichment model using both the document element and its image as input
class ExampleFormulaUnderstandingEnrichmentModel(BaseItemAndImageEnrichmentModel):
    images_scale = 2.6

    def __init__(self, enabled: bool):
        self.enabled = enabled

    def is_processable(self, doc: DoclingDocument, element: NodeItem) -> bool:
        return (
            self.enabled
            and isinstance(element, TextItem)
            and element.label == DocItemLabel.FORMULA
        )

    def __call__(
        self,
        doc: DoclingDocument,
        element_batch: Iterable[ItemAndImageEnrichmentElement],
    ) -> Iterable[NodeItem]:
        if not self.enabled:
            return

        for enrich_element in element_batch:
            enrich_element.image.show()

            yield enrich_element.item


# How the pipeline can be extended.
class ExampleFormulaUnderstandingPipeline(StandardPdfPipeline):
    def __init__(self, pipeline_options: ExampleFormulaUnderstandingPipelineOptions):
        super().__init__(pipeline_options)
        self.pipeline_options: ExampleFormulaUnderstandingPipelineOptions

        self.enrichment_pipe = [
            ExampleFormulaUnderstandingEnrichmentModel(
                enabled=self.pipeline_options.do_formula_understanding
            )
        ]

        if self.pipeline_options.do_formula_understanding:
            self.keep_backend = True

    @classmethod
    def get_default_options(cls) -> ExampleFormulaUnderstandingPipelineOptions:
        return ExampleFormulaUnderstandingPipelineOptions()
    

# Example main. In the final version, we simply have to set do_formula_understanding to true.
def main():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path("/home/akshay/projects/generativeAI/L05_AY2425.pdf")

    pipeline_options = ExampleFormulaUnderstandingPipelineOptions()
    pipeline_options.do_formula_understanding = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_cls=ExampleFormulaUnderstandingPipeline,
                pipeline_options=pipeline_options,
            )
        }
    )
    doc_converter.convert(input_doc_path)

In [15]:
if __name__ == "__main__":
    main()

INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document L05_AY2425.pdf
INFO:docling.document_converter:Finished converting document L05_AY2425.pdf in 16.13 sec.
